In [29]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.inspection import permutation_importance
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, classification_report
from scipy.stats import pointbiserialr

# airbnb = pd.read_csv("/content/Airbnb_Open_Data.csv", header=0)
airbnb = pd.read_csv("Airbnb_Open_Data.csv", header=0)
airbnb.head()
data_cleaned = airbnb.drop(['id', 'NAME', 'neighbourhood', 'house_rules', 'last review', 'host name', 'host id', 'lat', 'long', 'country',
                         'calculated host listings count', 'country code', 'license'], axis=1)
# 1. Handle missing values
# Fill or drop missing values based on context
data_cleaned.dropna()
# 1. Handle missing values
# Fill or drop missing values based on context
# 1. Handle missing values
# Fill or drop missing values based on context
data_cleaned.fillna({
    'host_identity_verified': 'unknown',  # Fill missing host verification as 'unknown'
    'reviews_per_month': 0               # Assume no reviews if data is missing
}, inplace=True)

# 2. Convert data types
# Parse dates and normalize data types
data_cleaned['price'] = data_cleaned['price'].replace(r'[\$,]', '', regex=True).astype(float)
data_cleaned['service fee'] = data_cleaned['service fee'].replace(r'[\$,]', '', regex=True).astype(float)

# 3. Normalize text fields
# Standardize text columns to lowercase and strip whitespace
text_columns = ['neighbourhood group', 'cancellation_policy', 'room type']
for col in text_columns:
    data_cleaned[col] = data_cleaned[col].str.lower().str.strip()

# 4. Encode categorical variables
# Use one-hot encoding or label encoding for categorical fields
categorical_columns = ['host_identity_verified', 'neighbourhood group', 'cancellation_policy', 'room type']
data_cleaned = pd.get_dummies(data_cleaned, columns=categorical_columns, drop_first=True)

# 5. Remove outliers
# Clip or filter out extreme values in numerical fields
price_threshold = data_cleaned['price'].quantile(0.99)
data_cleaned = data_cleaned[data_cleaned['price'] <= price_threshold]  # Keep values below 99th percentile

# 6. Scale numerical features
# Normalize numerical fields for training neural networks
from sklearn.preprocessing import MinMaxScaler

# List of expected numerical columns
numerical_columns = ['price', 'service fee', 'minimum nights', 'number of reviews', 'reviews_per_month', 'availability_365']

# Verify which columns are present in the DataFrame
existing_columns = [col for col in numerical_columns if col in data_cleaned.columns]

# Apply scaling only to existing columns
if existing_columns:
    scaler = MinMaxScaler()
    data_cleaned[existing_columns] = scaler.fit_transform(data_cleaned[existing_columns])
else:
    print("No numerical columns found in the dataset to scale.")

# Output to verify the changes
print(data_cleaned.head())


C:\Users\shiva\AppData\Local\Temp\ipykernel_14024\493862489.py:22: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  airbnb = pd.read_csv("Airbnb_Open_Data.csv", header=0)


  instant_bookable  Construction year     price  service fee  minimum nights  \
0            False             2020.0  0.804921     0.802632        0.062396   
1            False             2007.0  0.080844     0.078947        0.065724   
2             True             2005.0  0.500879     0.500000        0.061231   
3             True             2005.0  0.279438     0.280702        0.065724   
4            False             2009.0  0.135325     0.135965        0.062396   

   number of reviews  reviews per month  review rate number  availability 365  \
0           0.008789               0.21                 4.0             286.0   
1           0.043945               0.38                 4.0             228.0   
2           0.000000                NaN                 5.0             352.0   
3           0.263672               4.64                 4.0             322.0   
4           0.008789               0.10                 3.0             289.0   

   host_identity_verified_unknow

KeyError: "['reviews_per_month', 'availability_365'] not in index"

In [27]:
print(data_cleaned.columns)

Index(['instant_bookable', 'Construction year', 'price', 'service fee',
       'minimum nights', 'number of reviews', 'reviews per month',
       'review rate number', 'availability 365',
       'host_identity_verified_unknown', 'host_identity_verified_verified',
       'neighbourhood group_brookln', 'neighbourhood group_brooklyn',
       'neighbourhood group_manhatan', 'neighbourhood group_manhattan',
       'neighbourhood group_queens', 'neighbourhood group_staten island',
       'cancellation_policy_moderate', 'cancellation_policy_strict',
       'room type_hotel room', 'room type_private room',
       'room type_shared room'],
      dtype='object')


In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 1. Feature Extraction: Create total_price
data_cleaned['total_price'] = data_cleaned['price'] + data_cleaned['service fee']

# 2. Select Features (Independent Variables) for Model
# Select relevant features, for simplicity, let's start with the following columns:
features = [
    'minimum nights', 'number of reviews', 'reviews per month', 
    'availability 365', 'host_identity_verified_unknown', 
    'host_identity_verified_verified', 'neighbourhood group_brookln', 
    'neighbourhood group_brooklyn', 'neighbourhood group_manhatan',
    'neighbourhood group_manhattan', 'neighbourhood group_queens',
    'neighbourhood group_staten island', 'cancellation_policy_moderate', 
    'cancellation_policy_strict', 'room type_hotel room', 
    'room type_private room', 'room type_shared room'
]

# 3. Prepare the dataset for training
# We need to drop rows with missing target variables and encode categorical variables
data_cleaned.dropna(subset=['total_price'], inplace=True)  # Drop rows with missing target

# Prepare the features (X) and target (y)
X = data_cleaned[features]
y = data_cleaned['total_price']

# 4. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Train the Model (RandomForestRegressor)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 6. Evaluate the Model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# 7. Feature Importance (Optional, to understand which features are important)
feature_importances = pd.DataFrame(model.feature_importances_, index=features, columns=["importance"]).sort_values("importance", ascending=False)
print("Feature Importances:\n", feature_importances)

# 8. Rate Listings as 'Overpriced', 'Good Deal', or 'Bargain'
# For simplicity, we assume that the median of actual prices determines the classification
median_price = np.median(y_test)
listing_rating = []

for price in y_pred:
    if price > median_price * 1.2:
        listing_rating.append("Overpriced")
    elif price < median_price * 0.8:
        listing_rating.append("Bargain")
    else:
        listing_rating.append("Good Deal")

# 9. Add the ratings back to the test data
test_results = pd.DataFrame({
    'predicted_price': y_pred,
    'actual_price': y_test,
    'rating': listing_rating
})

print(test_results.head())

# Now you can use this data for further analysis or exporting


Mean Squared Error: 0.27808050065341217
R-squared: 0.16330858259027847
Feature Importances:
                                    importance
availability 365                     0.252618
reviews per month                    0.245642
number of reviews                    0.173607
minimum nights                       0.116769
host_identity_verified_verified      0.035765
room type_private room               0.034378
cancellation_policy_strict           0.031361
cancellation_policy_moderate         0.031177
neighbourhood group_brooklyn         0.025218
neighbourhood group_manhattan        0.024044
neighbourhood group_queens           0.018096
room type_shared room                0.006335
neighbourhood group_staten island    0.003213
host_identity_verified_unknown       0.001267
room type_hotel room                 0.000509
neighbourhood group_brookln          0.000002
neighbourhood group_manhatan         0.000000
       predicted_price  actual_price      rating
1079          1.014575      0.